# Measurement Errors in Linear Regression
### January 6, 2020

[Brigitta Sipőcz, University of Washington](https://bsipocz.github.io/) 

Resources for this notebook:

- [Textbook](http://press.princeton.edu/titles/10159.html) Chapter 8.
- Paper by [B Kelly, ApJ, 665, 2007](https://iopscience.iop.org/article/10.1086/519947/fulltext/70991.html)
- [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas

We rely on scikit-learn, astroML and pymc3. Full functionality will be available in the next release of astroML (v1.0)

### Lets use simulated data here. 
First we will model the distance of 100 supernovas (for a particular cosmology) as a function of redshift.

We rely on that astroML has a common API with scikit-learn, extending the functionality of the latter.



In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from astropy.cosmology import LambdaCDM
from astroML.datasets import generate_mu_z

z_sample, mu_sample, dmu = generate_mu_z(100, random_state=0)

cosmo = LambdaCDM(H0=70, Om0=0.30, Ode0=0.70, Tcmb0=0)
z = np.linspace(0.01, 2, 1000)
mu_true = cosmo.distmod(z)

plt.errorbar(z_sample, mu_sample, dmu, fmt='.')

## Simple linear regression

Regression defined as the relation between a dependent variable, $y$, and a set of independent variables, $x$, 
that describes the expectation value of y given x: $ E[y|x] $

We will start with the most familiar linear regression, a straight-line fit to data.
A straight-line fit is a model of the form
$$
y = ax + b
$$
where $a$ is commonly known as the *slope*, and $b$ is commonly known as the *intercept*.

We can use Scikit-Learn's LinearRegression estimator to fit this data and construct the best-fit line:

In [ ]:
from sklearn.linear_model import LinearRegression as LinearRegression_sk 

linear_sk = LinearRegression_sk()
linear_sk.fit(z_sample[:,None], mu_sample)

mu_fit_sk = linear_sk.predict(z[:, None])

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(8, 6))

ax = fig.add_subplot(111)

ax.plot(z, mu_fit_sk, '-k')
ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, fmt='.k', ecolor='gray', lw=1)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

plt.show()

## Measurement errors

Modifications to LinearRegression in astroML take measurement errors into account on the dependent variable.

In [ ]:
from astroML.linear_model import LinearRegression

linear = LinearRegression()
linear.fit(z_sample[:,None], mu_sample, dmu)

mu_fit = linear.predict(z[:, None])

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(8, 6))

ax = fig.add_subplot(111)

ax.plot(z, mu_fit_sk, '-k')
ax.plot(z, mu_fit, '-k', color='red')

ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, fmt='.k', ecolor='gray', lw=1)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

plt.show()

## Basis function regression

If we consider a function in terms of the sum of bases (this can be polynomials, Gaussians, quadratics, cubics) then we can solve for the coefficients using regression. 

### Polynomial basis functions

polynomial regression: $$𝑦=𝑎_0+𝑎_1𝑥+𝑎_2𝑥^2+𝑎_3𝑥^3+⋯$$

Notice that this is still a linear model—the linearity refers to the fact that the coefficients $𝑎_𝑛$ never multiply or divide each other. What we have effectively done here is to take our one-dimensional $𝑥$ values and projected them into a higher dimension, so that a linear fit can fit more complicated relationships between $𝑥$ and $𝑦$.

In [ ]:
from astroML.linear_model import PolynomialRegression

# 2nd degree polynomial regression
polynomial = PolynomialRegression(degree=2)
polynomial.fit(z_sample[:,None], mu_sample, dmu)

mu_fit_poly = polynomial.predict(z[:, None])

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(8, 6))

ax = fig.add_subplot(111)

ax.plot(z, mu_fit_poly, '-k', color='red')

ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, fmt='.k', ecolor='gray', lw=1)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

plt.show()

### Gaussian basis functions

Of course, other basis functions are possible.
For example, one useful pattern is to fit a model that is not a sum of polynomial bases, but a sum of Gaussian bases. E.g. we could substitute $𝑥^2$ for Gaussians (where we fix $𝜎$ and $𝜇$ and fit for the amplitude) as long as the attribute we are fitting for is linear. This is called basis function regression.


In [ ]:
from astroML.linear_model import BasisFunctionRegression

#------------------------------------------------------------
# Define our number of Gaussians
nGaussians = 20
basis_mu = np.linspace(0, 2, nGaussians)[:, None]
basis_sigma = 3 * (basis_mu[1] - basis_mu[0])

gauss_basis = BasisFunctionRegression('gaussian', mu=basis_mu, sigma=basis_sigma)
gauss_basis.fit(z_sample[:,None], mu_sample, dmu)

mu_fit_gauss = gauss_basis.predict(z[:, None])

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(8, 6))

ax = fig.add_subplot(111)

ax.plot(z, mu_fit_gauss, '-k', color='red')

ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, fmt='.k', ecolor='gray', lw=1)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

plt.show()

### Exercise - how many polynomial terms (or Gaussians do we need to best fit the data)

Increase the number of terms that we use to fit the data and plot the results until you are happy. Calculate the rms error (between the data `mu_sample` and the fit `mu_sample_fit`)

Generate another set of input data and apply the model to these data. Is the rms the same? Why?

### Regularization and cross-validation

As the complexity of a model increases the data points fit the model more and more closely.

This does not result in a better fit to the data. We are overfitting the data (the model has high variance - a small change in a training point can change the model dramatically).

We can evaluate this using a training set (50-70% of sample), a cross-validation set (15-25%) and a test set (15-25%). See book sub-chapter 8.11 and e.g. figures [8.13](http://www.astroml.org/book_figures/chapter8/fig_cross_val_B.html) and [8.14](http://www.astroml.org/book_figures/chapter8/fig_cross_val_C.html).

For cases where we are concerned with overfitting we can apply constraints (usually of smoothness, number of coefficients, size of coefficients). See book sub-chapter 8.3, and e.g. figure [8.4](http://www.astroml.org/book_figures/chapter8/fig_rbf_ridge_mu_z.html)



# Measurement errors in both dependent and independent variables

Use simulation data from [Kelly 2007](https://iopscience.iop.org/article/10.1086/519947/pdf) where there is measurement error on the observed values $x_i$ and $y_i$ as well as intrinsic scatter in the regression relationship: $$ \eta_i = \alpha + \beta \xi_i + \epsilon_i $$ and $$ x_i = \xi_i + \epsilon_{x,i}$$ $$y_i = \eta_i + \epsilon_{y,i}$$


In [ ]:
from simulator_kelly import simulation

ksi, eta, xi, yi, xi_error, yi_error, alpha_in, beta_in = simulation(size=100, scalex=0.2, scaley=0.2,
                                                                     alpha=2, beta=[0.5,])

### LinearRegressionwithErrors will be part of the next astroML release, v1.0
Change this import in the cell below to:

`from astroML.linear_model import LinearRegressionwithErrors`

In [ ]:
from linear_model import LinearRegressionwithErrors

In [ ]:
linreg_xy_err = LinearRegressionwithErrors()
linreg_xy_err.fit(xi, yi, yi_error, xi_error)

## Some plotting functions we use below

In [ ]:
def plot_figure(ksi, eta, x, y, sigma_x, sigma_y, add_regression_lines=False,
                alpha_in=1, beta_in=0.5):

    figure = plt.figure(figsize=(15, 6))

    ax = figure.add_subplot(121)
    ax.scatter(x, y, alpha=0.5)
    ax.errorbar(x, y, xerr=sigma_x, yerr=sigma_y, alpha=0.3, ls='')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    # True regression line
    x0 = np.linspace(-10, 10, 40)
    y0 = alpha_in + x0 * beta_in

    ax.plot(x0, y0, color='black', label='True')
    ax.set_xlim(-12, 12)
    ax.set_ylim(-5, 7)

    ax.legend()
    
    
def plot_trace(traces, observed, ax=None, chains=None, multidim_ind=None):

    xi, yi, sigx, sigy = observed

    if multidim_ind is not None:
        xi = xi[multidim_ind]

    x = np.linspace(np.min(xi)-0.5, np.max(xi)+0.5, 50)

    for i, trace in enumerate(traces):
        if 'theta' in trace.varnames and 'slope' not in trace.varnames:
            trace.add_values({'slope': np.tan(trace['theta'])})

        if multidim_ind is not None:
            trace_slope = trace['slope'][:, multidim_ind]
        else:
            trace_slope = trace['slope'][:, 0]

        if chains is not None:
            for chain in range(100, len(trace) * trace.nchains, chains):
                y = trace['inter'][chain] + trace_slope[chain] * x
                ax.plot(x, y, alpha=0.03, c='red')

        # plot the best-fit line only
        H2D, bins1, bins2 = np.histogram2d(trace_slope,
                                           trace['inter'], bins=50)

        w = np.where(H2D == H2D.max())

        # choose the maximum posterior slope and intercept
        slope_best = bins1[w[0][0]]
        intercept_best = bins2[w[1][0]]

        print("beta:", slope_best, "alpha:", intercept_best)
        y = intercept_best + slope_best * x
        
        ax.plot(x, y, ':', label='fitted')
        
        ax.legend()
        break
        

In [ ]:
plot_figure(ksi, eta, xi, yi, xi_error, yi_error, add_regression_lines=False, alpha_in=alpha_in, beta_in=beta_in)
plot_trace([linreg_xy_err.trace,], (xi, yi, xi_error, yi_error), ax=plt.gca(), chains=50)


## Multivariate regression
For multivariate data (where we fit a hyperplane rather than a straight line) we simply extend the description of the regression function to multiple dimensions.

In [ ]:
from simulator_kelly import simulation_multidim

ksi, eta, xi, yi, xi_error, yi_error, alpha_in, beta_in = simulation_multidim(size=100, scalex=0.2, scaley=0.2,
                                                                              alpha=2, beta=np.array((0.5, 1, 1)),
                                                                              multidim=3)

In [ ]:
linreg_xy_err = LinearRegressionwithErrors()
linreg_xy_err.fit(xi, yi, yi_error, xi_error)

In [ ]:
for i in range(linreg_xy_err.trace['slope'].shape[1]):

    joinpl = sns.jointplot(linreg_xy_err.trace['slope'][:, i], linreg_xy_err.trace['inter'], kind='kde')
    joinpl.ax_joint.plot(beta_in[i], alpha_in, 'x', color='red', ms=10)
    joinpl.ax_marg_y.plot([0, 2], [alpha_in, alpha_in], color='red')
    joinpl.ax_marg_x.plot([beta_in[i], beta_in[i]], [0, 2], color='red')
    
    plot_figure(ksi[i], eta, xi[i], yi, xi_error[i], yi_error, add_regression_lines=False, alpha_in=alpha_in, beta_in=beta_in[i])
    plot_trace([linreg_xy_err.trace,], (xi, yi, xi_error, yi_error), ax=plt.gca(), chains=50, multidim_ind=i)
